In [7]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import re
import time

# ChromeDriver 설정
service = Service(ChromeDriverManager().install())
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=options)

url = 'https://map.naver.com/v5/search'
driver.get(url)
key_word = '강남구 치킨'  # 검색어

# CSS 요소를 기다리는 함수
def time_wait(num, code):
    try:
        wait = WebDriverWait(driver, num).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, code)))
    except TimeoutException:
        print(code, '태그를 찾지 못하였습니다.')
        driver.quit()
    return wait

# 검색 입력 상자가 나타날 때까지 대기
time_wait(5, 'div.input_box > input.input_search')

# 검색 상자를 찾아 검색어 입력
search = driver.find_element(By.CSS_SELECTOR, 'div.input_box > input.input_search')
search.send_keys(key_word)  # 검색어 입력
search.send_keys(Keys.ENTER)  # 엔터키 누르기

time.sleep(1)

# 특정 프레임으로 전환하는 함수
def switch_frame(frame_id):
    driver.switch_to.default_content()  # 기본 컨텐츠로 리셋
    frame = driver.find_element(By.ID, frame_id)
    driver.switch_to.frame(frame)  # 지정된 프레임으로 전환

# 검색 결과 프레임으로 전환
switch_frame('searchIframe')

# 페이지 다운 함수
def page_down(num):
    body = driver.find_element(By.CSS_SELECTOR, 'body')
    body.click()
    for _ in range(num):
        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(0.4)  # 각 페이지 다운 사이에 약간의 지연 추가

# 더 많은 결과를 로드하기 위해 페이지 다운
page_down(22)
time.sleep(5)

# 매장 리스트 컨테이너
store_list = driver.find_elements(By.CSS_SELECTOR, 'li.UEzoS')[0:20]  # 상위 70개 매장만 선택

# 결과를 저장할 딕셔너리
store_dict = {'매장정보': []}
# 스크립트 시작 시간
start = time.time()
print('[크롤링 시작...]')

# 크롤링 (상위 20개 가게만)
for store in range(len(store_list)):
    try:    
        # 상세페이지 태그 요소를 찾아 클릭
        link_element = store_list[store].find_element(By.CSS_SELECTOR, 'a.WwIU2')
        link_element.click()
        time.sleep(1)
        switch_frame('entryIframe')
        time_wait(5, '.GHAhO')

        # 상세 페이지 조금만 스크롤
        driver.execute_script("window.scrollBy(0, 750);")
        time.sleep(5)

        # 매장명 추출
        store_name = driver.find_element(By.CSS_SELECTOR, '.GHAhO').text

        # 최신순 댓글 정렬 버튼 찾기 및 클릭
        review_sort_button = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[1]/div[2]/div[1]/span[2]/a')
        review_sort_button.click()
        time.sleep(5)

         # 리뷰 더보기 클릭
        try:
            for _ in range(33):
                more_review_buttons = driver.find_elements(By.CSS_SELECTOR, 'a.fvwqf')
                for button in more_review_buttons:
                    button.click()
                    time.sleep(1.1)
        except:
            pass

        # 리뷰 추출
        reviews = []
        try:
            review_list = driver.find_elements(By.CSS_SELECTOR, 'span.zPfVt')[:1000]  # 리뷰 텍스트만 상위 최대 1000개 가져오기
            for review in review_list:
                review_text = review.text  # 리뷰 텍스트
                reviews.append(review_text)
        except:
            reviews = []

        # ---- dict에 데이터 집어넣기----
        dict_temp = {
            'name': store_name,
            'reviews': reviews
        }

        store_dict['매장정보'].append(dict_temp)

        print(f'{store_name} 매장 완료')
        switch_frame('searchIframe')
        time.sleep(1)

    except Exception as e:
        print(f'ERROR! {e}')

print('[데이터 수집 완료]\n소요 시간 :', time.time() - start)
driver.quit()  # 작업이 끝나면 창을 닫는다.


[크롤링 시작...]
삼성동치킨 삼성점 매장 완료
나이티지 매장 완료
윙스탑 강남점 매장 완료
현정치킨바베큐 3호점 매장 완료
[데이터 수집 완료]
소요 시간 : 242.94734835624695


In [8]:
import json
# json 파일로 저장
with open('C:/Users/maind/semipro/store_data_fin1.json', 'w', encoding='utf-8') as f:
    json.dump(store_dict, f, indent=4, ensure_ascii=False)

import pandas as pd

naver_json = open('C:/Users/maind/semipro/store_data_fin1.json', encoding = 'utf-8')
naver_dict = json.load(naver_json)   #=> 파이썬 자료형(딕셔너리나 리스트)으로 반환
df1 = pd.DataFrame(naver_dict)
df2 = pd.DataFrame(naver_dict)

# csv파일로 저장

file_path1 = 'C:/Users/maind/semipro/store_data_fin1_csv.csv'
df1.to_csv(file_path1, index=False)

# csv로 변경

import pandas as pd

# 데이터프레임 생성
df1 = pd.DataFrame(store_dict['매장정보'])

# 'reviews' 리스트를 개별 리뷰로 분리하여 행을 확장
df_expanded1 = df1.explode('reviews')

# 결과 출력
file_path1 = 'C:/Users/maind/semipro/store_data_fin1_csv.csv'
df_expanded1.to_csv(file_path1, index = False)